In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16, MobileNetV2
from tensorflow.keras.models import Sequential, Model
from sklearn.metrics import confusion_matrix

# Helper libraries
import numpy as np
import seaborn as sns
import matplotlib
matplotlib.use('Agg')  # Use 'TkAgg' if you want to display plots in a separate window
import matplotlib.pyplot as plt
import PIL
from sklearn.model_selection import KFold
from tqdm import tqdm
import os

# New imports for model visualization
import graphviz
import pydot
from tensorflow.keras.utils import plot_model
from IPython.display import display, Image


In [2]:
# Load the dataset
train_dir = r"D:\University Materials\SM 5\Adcanved AI\seg_train\seg_train"
test_dir = r"D:\University Materials\SM 5\Adcanved AI\seg_test\seg_test"
class_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]

In [3]:
# Define image size and batch size
img_height = 150
img_width = 150
batch_size = 32

In [4]:
# Create data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="sparse",
    subset="training"
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="sparse",
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="sparse"
)

Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [5]:
# Define model architectures
def create_basic_cnn():
    model = Sequential([
        layers.Conv2D(32, 3, activation="relu", input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D(2),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(2),
        layers.Conv2D(128, 3, activation="relu"),
        layers.MaxPooling2D(2),
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dense(6, activation="softmax")
    ])
    return model

def create_advanced_cnn():
    model = Sequential([
        layers.Conv2D(32, 3, activation="relu", input_shape=(img_height, img_width, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        layers.Conv2D(64, 3, activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        layers.Conv2D(128, 3, activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(6, activation="softmax")
    ])
    return model

def create_transfer_learning_model(base_model):
    for layer in base_model.layers:
        layer.trainable = False
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    output = layers.Dense(6, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    return model

def create_resnet50_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    return create_transfer_learning_model(base_model)

def create_vgg16_model():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    return create_transfer_learning_model(base_model)

def create_mobilenetv2_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    return create_transfer_learning_model(base_model)

In [6]:
# Define K-fold cross-validation
n_splits = 5
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

In [7]:
def train_and_evaluate_model_cv(model_func, model_name):
    cv_scores = []
    
    for fold, (train_indices, val_indices) in enumerate(kfold.split(range(len(train_generator)))):
        print(f"Training {model_name} - Fold {fold + 1}/{n_splits}")
        
        # Create a new model for each fold
        model = model_func()
        model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        
        # Create data generators for this fold
        train_data = train_datagen.flow_from_directory(
            train_dir,
            target_size=(img_height, img_width),
            batch_size=batch_size,
            class_mode="sparse",
            subset="training",
            shuffle=True,
            seed=42
        )
        val_data = train_datagen.flow_from_directory(
            train_dir,
            target_size=(img_height, img_width),
            batch_size=batch_size,
            class_mode="sparse",
            subset="validation",
            shuffle=True,
            seed=42
        )
        
        # Train the model
        history = model.fit(
            train_data,
            epochs=30,
            validation_data=val_data,
            callbacks=[
                keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
                keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3)
            ],
            verbose=1
        )
        
        # Evaluate the model
        val_loss, val_accuracy = model.evaluate(val_data)
        cv_scores.append(val_accuracy)
        
        print(f"Fold {fold + 1} - Validation Accuracy: {val_accuracy:.4f}")
        
        # Save the model
        model.save(f'model_{model_name}_fold_{fold+1}.h5')
    
    print(f"{model_name} - Average Validation Accuracy: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
    
    # Train on full training data and evaluate on test data
    model = model_func()
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    history = model.fit(
        train_generator,
        epochs=30,
        validation_data=validation_generator,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3)
        ]
    )
    test_loss, test_accuracy = model.evaluate(test_generator)
    
    # Save the final model
    model.save(f'model_{model_name}_final.h5')
    
    return history, np.mean(cv_scores), np.std(cv_scores), test_accuracy


In [8]:
# Train and evaluate all models
results = {}

for model_name, model_func in {
    "Basic CNN": create_basic_cnn,
    "Advanced CNN": create_advanced_cnn,
    "ResNet50": create_resnet50_model,
    "VGG16": create_vgg16_model,
    "MobileNetV2": create_mobilenetv2_model
}.items():
    print(f"\nTraining {model_name}...")
    try:
        history, cv_mean, cv_std, test_accuracy = train_and_evaluate_model_cv(model_func, model_name)
        results[model_name] = {
            "history": history,
            "cv_mean": cv_mean,
            "cv_std": cv_std,
            "test_accuracy": test_accuracy
        }
        print(f"{model_name} - Test Accuracy: {test_accuracy:.4f}")
    except Exception as e:
        print(f"An error occurred while training {model_name}: {str(e)}")



Training Basic CNN...
Training Basic CNN - Fold 1/5


c:\users\abdul\appdata\local\programs\python\python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Epoch 1/30


c:\users\abdul\appdata\local\programs\python\python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


351/351 ━━━━━━━━━━━━━━━━━━━━ 323s 899ms/step - accuracy: 0.4750 - loss: 1.3281 - val_accuracy: 0.6787 - val_loss: 0.8644 - learning_rate: 0.0010
Epoch 2/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 237s 670ms/step - accuracy: 0.6936 - loss: 0.8298 - val_accuracy: 0.7318 - val_loss: 0.7320 - learning_rate: 0.0010
Epoch 3/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 246s 694ms/step - accuracy: 0.7332 - loss: 0.7238 - val_accuracy: 0.7354 - val_loss: 0.7293 - learning_rate: 0.0010
Epoch 4/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 246s 694ms/step - accuracy: 0.7412 - loss: 0.6930 - val_accuracy: 0.7785 - val_loss: 0.6290 - learning_rate: 0.0010
Epoch 5/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 245s 692ms/step - accuracy: 0.7755 - loss: 0.6278 - val_accuracy: 0.7411 - val_loss: 0.7036 - learning_rate: 0.0010
Epoch 6/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 246s 693ms/step - accuracy: 0.7788 - loss: 0.5882 - val_accuracy: 0.7468 - val_loss: 0.7010 - learning_rate: 0.0010
Epoch 7/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 251s 709ms/step - accuracy: 0.7914 

...

Fold 2 - Validation Accuracy: 0.8880
Training MobileNetV2 - Fold 3/5
Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Epoch 1/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 134s 369ms/step - accuracy: 0.7759 - loss: 0.6458 - val_accuracy: 0.8584 - val_loss: 0.3771 - learning_rate: 0.0010
Epoch 2/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 365ms/step - accuracy: 0.8605 - loss: 0.3694 - val_accuracy: 0.8684 - val_loss: 0.3552 - learning_rate: 0.0010
Epoch 3/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 128s 363ms/step - accuracy: 0.8716 - loss: 0.3305 - val_accuracy: 0.8655 - val_loss: 0.3591 - learning_rate: 0.0010
Epoch 4/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 364ms/step - accuracy: 0.8836 - loss: 0.3169 - val_accuracy: 0.8641 - val_loss: 0.3550 - learning_rate: 0.0010
Epoch 5/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 128s 363ms/step - accuracy: 0.8880 - loss: 0.3088 - val_accuracy: 0.8738 - val_loss: 0.3316 - learning_rate: 0.0010
Epoch 6/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 366ms/step - accuracy

Fold 3 - Validation Accuracy: 0.8873
Training MobileNetV2 - Fold 4/5
Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Epoch 1/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 134s 369ms/step - accuracy: 0.7549 - loss: 0.6780 - val_accuracy: 0.8581 - val_loss: 0.3936 - learning_rate: 0.0010
Epoch 2/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 364ms/step - accuracy: 0.8606 - loss: 0.3804 - val_accuracy: 0.8545 - val_loss: 0.4143 - learning_rate: 0.0010
Epoch 3/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 132s 372ms/step - accuracy: 0.8716 - loss: 0.3503 - val_accuracy: 0.8720 - val_loss: 0.3330 - learning_rate: 0.0010
Epoch 4/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 128s 363ms/step - accuracy: 0.8836 - loss: 0.3215 - val_accuracy: 0.8755 - val_loss: 0.3450 - learning_rate: 0.0010
Epoch 5/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 365ms/step - accuracy: 0.8864 - loss: 0.3020 - val_accuracy: 0.8777 - val_loss: 0.3310 - learning_rate: 0.0010
Epoch 6/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 364ms/step - accuracy

Fold 4 - Validation Accuracy: 0.8852
Training MobileNetV2 - Fold 5/5
Found 11230 images belonging to 6 classes.
Found 2804 images belonging to 6 classes.
Epoch 1/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 135s 371ms/step - accuracy: 0.7622 - loss: 0.6466 - val_accuracy: 0.8581 - val_loss: 0.3735 - learning_rate: 0.0010
Epoch 2/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 130s 367ms/step - accuracy: 0.8577 - loss: 0.3840 - val_accuracy: 0.8684 - val_loss: 0.3721 - learning_rate: 0.0010
Epoch 3/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 364ms/step - accuracy: 0.8756 - loss: 0.3407 - val_accuracy: 0.8620 - val_loss: 0.3621 - learning_rate: 0.0010
Epoch 4/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 365ms/step - accuracy: 0.8769 - loss: 0.3313 - val_accuracy: 0.8634 - val_loss: 0.3540 - learning_rate: 0.0010
Epoch 5/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 365ms/step - accuracy: 0.8834 - loss: 0.3226 - val_accuracy: 0.8816 - val_loss: 0.3449 - learning_rate: 0.0010
Epoch 6/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 129s 364ms/step - accuracy

Fold 5 - Validation Accuracy: 0.8787
MobileNetV2 - Average Validation Accuracy: 0.8847 (+/- 0.0033)
Epoch 1/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 150s 415ms/step - accuracy: 0.7677 - loss: 0.6560 - val_accuracy: 0.8673 - val_loss: 0.3640 - learning_rate: 0.0010
Epoch 2/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 137s 387ms/step - accuracy: 0.8582 - loss: 0.3750 - val_accuracy: 0.8645 - val_loss: 0.3718 - learning_rate: 0.0010
Epoch 3/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 133s 376ms/step - accuracy: 0.8646 - loss: 0.3491 - val_accuracy: 0.8524 - val_loss: 0.3895 - learning_rate: 0.0010
Epoch 4/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 135s 381ms/step - accuracy: 0.8810 - loss: 0.3295 - val_accuracy: 0.8659 - val_loss: 0.3550 - learning_rate: 0.0010
Epoch 5/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 134s 378ms/step - accuracy: 0.8818 - loss: 0.3080 - val_accuracy: 0.8666 - val_loss: 0.3593 - learning_rate: 0.0010
Epoch 6/30
351/351 ━━━━━━━━━━━━━━━━━━━━ 132s 373ms/step - accuracy: 0.8870 - loss: 0.2949 - val_accuracy: 0.8684 - val_l

MobileNetV2 - Test Accuracy: 0.9017


In [6]:
# Visualization functions
def plot_training_history(results):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
    
    for model_name, result in results.items():
        ax1.plot(result['history'].history['accuracy'], label=f'{model_name} (Train)')
        ax1.plot(result['history'].history['val_accuracy'], label=f'{model_name} (Val)')
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend()
    
    for model_name, result in results.items():
        ax2.plot(result['history'].history['loss'], label=f'{model_name} (Train)')
        ax2.plot(result['history'].history['val_loss'], label=f'{model_name} (Val)')
    ax2.set_title('Model Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

def plot_test_accuracy(results):
    model_names = list(results.keys())
    test_accuracies = [result['test_accuracy'] for result in results.values()]
    cv_means = [result['cv_mean'] for result in results.values()]
    cv_stds = [result['cv_std'] for result in results.values()]
    
    x = np.arange(len(model_names))
    width = 0.35
    
    fig, ax = plt.subplots(figsize=(12, 6))
    rects1 = ax.bar(x - width/2, test_accuracies, width, label='Test Accuracy')
    rects2 = ax.bar(x + width/2, cv_means, width, label='CV Mean Accuracy')
    
    ax.set_ylabel('Accuracy')
    ax.set_title('Model Accuracies')
    ax.set_xticks(x)
    ax.set_xticklabels(model_names, rotation=45, ha='right')
    ax.legend()
    
    ax.bar_label(rects1, padding=3, fmt='%.4f')
    ax.bar_label(rects2, padding=3, fmt='%.4f')
    
    # Add error bars for CV
    ax.errorbar(x + width/2, cv_means, yerr=cv_stds, fmt='none', capsize=5, color='black')
    
    fig.tight_layout()
    plt.savefig('model_accuracies.png')
    plt.close()

def plot_confusion_matrices(results, test_generator):
    n_models = len(results)
    fig, axes = plt.subplots(n_models, 1, figsize=(8, 6*n_models))
    if n_models == 1:
        axes = [axes]
    
    for ax, (model_name, result) in zip(axes, results.items()):
        model = keras.models.load_model(f'model_{model_name}_final.h5')
        y_pred = model.predict(test_generator)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = test_generator.classes
        
        cm = confusion_matrix(y_true, y_pred_classes)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
        ax.set_title(f'Confusion Matrix - {model_name}')
        ax.set_xlabel('Predicted')
        ax.set_ylabel('True')
    
    plt.tight_layout()
    plt.savefig('confusion_matrices.png')
    plt.close()

def plot_learning_curves(results):
    fig, axes = plt.subplots(len(results), 2, figsize=(15, 5*len(results)))
    
    for i, (model_name, result) in enumerate(results.items()):
        history = result['history'].history
        
        axes[i, 0].plot(history['accuracy'], label='Train Accuracy')
        axes[i, 0].plot(history['val_accuracy'], label='Validation Accuracy')
        axes[i, 0].set_title(f'{model_name} - Accuracy')
        axes[i, 0].set_xlabel('Epoch')
        axes[i, 0].set_ylabel('Accuracy')
        axes[i, 0].legend()
        
        axes[i, 1].plot(history['loss'], label='Train Loss')
        axes[i, 1].plot(history['val_loss'], label='Validation Loss')
        axes[i, 1].set_title(f'{model_name} - Loss')
        axes[i, 1].set_xlabel('Epoch')
        axes[i, 1].set_ylabel('Loss')
        axes[i, 1].legend()
    
    plt.tight_layout()
    plt.savefig('learning_curves.png')
    plt.close()

def plot_model_comparison(results):
    model_names = list(results.keys())
    test_accuracies = [result['test_accuracy'] for result in results.values()]
    cv_means = [result['cv_mean'] for result in results.values()]
    cv_stds = [result['cv_std'] for result in results.values()]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(model_names))
    
    ax.errorbar(x, cv_means, yerr=cv_stds, fmt='o', label='CV Accuracy', capsize=5)
    ax.scatter(x, test_accuracies, color='red', label='Test Accuracy')
    
    ax.set_xlabel('Model')
    ax.set_ylabel('Accuracy')
    ax.set_title('Model Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels(model_names, rotation=45, ha='right')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig('model_comparison.png')
    plt.close()

def plot_model_architecture(model, model_name):
    # Create a dot graph
    dot = graphviz.Digraph(comment='Model Architecture')
    dot.attr(rankdir='TB', size='8,8')

    # Add nodes for each layer
    for i, layer in enumerate(model.layers):
        # Get layer class name
        layer_type = layer.__class__.__name__
        
        # Get output shape
        if hasattr(layer, 'output_shape'):
            if isinstance(layer.output_shape, tuple):
                shape = str(layer.output_shape)
            elif isinstance(layer.output_shape, list):
                shape = str(layer.output_shape[0])
            else:
                shape = "Unknown"
        else:
            try:
                shape = str(layer.get_output_at(0).shape)
            except:
                shape = "Unknown"
        
        # Create node label
        label = f"{layer_type}\n{shape}"
        
        dot.node(f'layer_{i}', label)
        
        # Add edges
        if i > 0:
            dot.edge(f'layer_{i-1}', f'layer_{i}')

    # Save the graph
    dot.render(f'model_architecture_{model_name}', format='png', cleanup=True)
    print(f"Model architecture saved as model_architecture_{model_name}.png")

    # Also use Keras' built-in plot_model function
    plot_model(model, to_file=f'model_architecture_{model_name}_keras.png', show_shapes=True, show_layer_names=True)
    print(f"Keras model architecture saved as model_architecture_{model_name}_keras.png")



In [7]:
# Visualize results
plot_training_history(results)
plot_test_accuracy(results)
plot_confusion_matrices(results, test_generator)
plot_learning_curves(results)
plot_model_comparison(results)


NameError: name 'results' is not defined

In [8]:
# Print final results
print("\nFinal Results:")
for model_name, result in results.items():
    print(f"{model_name}:")
    print(f"  CV Accuracy: {result['cv_mean']:.4f} (+/- {result['cv_std']:.4f})")
    print(f"  Test Accuracy: {result['test_accuracy']:.4f}")


Final Results:


NameError: name 'results' is not defined

In [17]:
# Print current working directory
print(f"Current working directory: {os.getcwd()}")

# Directory where your current models are saved
current_models_dir = '.'  # Assuming they're in the current directory

# Directory to save new format models
save_dir = 'saved_models_keras_format'
os.makedirs(save_dir, exist_ok=True)

# Function to save a model in Keras format
def save_model_keras_format(model, model_name):
    file_path = os.path.join(save_dir, f'{model_name}.keras')
    model.save(file_path)
    print(f"Model saved to {file_path}")

# Model names
model_names = ["Basic CNN", "Advanced CNN", "ResNet50", "VGG16", "MobileNetV2"]

# Number of folds
n_splits = 5

# Process final models and fold models
for model_name in model_names:
    # Process final model
    final_model_filename = f'model_{model_name}_final.h5'
    final_model_filepath = os.path.join(current_models_dir, final_model_filename)
    
    if os.path.exists(final_model_filepath):
        try:
            model = tf.keras.models.load_model(final_model_filepath)
            print(f"Loaded final model: {final_model_filename}")
            save_model_keras_format(model, f"{model_name.replace(' ', '_')}_final")
        except Exception as e:
            print(f"Error processing final model {model_name}: {str(e)}")
    else:
        print(f"Final model file not found: {final_model_filename}")
    
    # Process fold models
    for fold in range(1, n_splits + 1):
        fold_model_filename = f'model_{model_name}_fold_{fold}.h5'
        fold_model_filepath = os.path.join(current_models_dir, fold_model_filename)
        
        if os.path.exists(fold_model_filepath):
            try:
                model = tf.keras.models.load_model(fold_model_filepath)
                print(f"Loaded fold model: {fold_model_filename}")
                save_model_keras_format(model, f"{model_name.replace(' ', '_')}_fold_{fold}")
            except Exception as e:
                print(f"Error processing fold model {model_name} fold {fold}: {str(e)}")
        else:
            print(f"Fold model file not found: {fold_model_filename}")

print("All models processed.")

Current working directory: C:\Users\abdul
Loaded final model: model_Basic CNN_final.h5


Model saved to saved_models_keras_format\Basic_CNN_final.keras
Loaded fold model: model_Basic CNN_fold_1.h5


Model saved to saved_models_keras_format\Basic_CNN_fold_1.keras
Loaded fold model: model_Basic CNN_fold_2.h5


Model saved to saved_models_keras_format\Basic_CNN_fold_2.keras
Loaded fold model: model_Basic CNN_fold_3.h5


Model saved to saved_models_keras_format\Basic_CNN_fold_3.keras
Loaded fold model: model_Basic CNN_fold_4.h5


...

Loaded fold model: model_MobileNetV2_fold_1.h5
Model saved to saved_models_keras_format\MobileNetV2_fold_1.keras


Loaded fold model: model_MobileNetV2_fold_2.h5
Model saved to saved_models_keras_format\MobileNetV2_fold_2.keras


Loaded fold model: model_MobileNetV2_fold_3.h5
Model saved to saved_models_keras_format\MobileNetV2_fold_3.keras


Loaded fold model: model_MobileNetV2_fold_4.h5
Model saved to saved_models_keras_format\MobileNetV2_fold_4.keras


Loaded fold model: model_MobileNetV2_fold_5.h5
Model saved to saved_models_keras_format\MobileNetV2_fold_5.keras
All models processed.


In [10]:
print("\nGenerating model architecture diagrams...")
for model_name, result in results.items():
    try:
        model_path = os.path.join(save_dir, f'{model_name.replace(" ", "_")}_final.keras')
        model = keras.models.load_model(model_path)
        
        # Ensure the model is built
        if not model.built:
            dummy_data = np.random.random((1, img_height, img_width, 3))
            model.predict(dummy_data)
        
        plot_model_architecture(model, model_name)
    except Exception as e:
        print(f"Error processing model {model_name}: {str(e)}")


Generating model architecture diagrams...
Model architecture saved as model_architecture_Basic CNN.png
Keras model architecture saved as model_architecture_Basic CNN_keras.png
Model architecture saved as model_architecture_Advanced CNN.png
Keras model architecture saved as model_architecture_Advanced CNN_keras.png
Model architecture saved as model_architecture_ResNet50.png
Keras model architecture saved as model_architecture_ResNet50_keras.png
Model architecture saved as model_architecture_VGG16.png
Keras model architecture saved as model_architecture_VGG16_keras.png
Model architecture saved as model_architecture_MobileNetV2.png
Keras model architecture saved as model_architecture_MobileNetV2_keras.png
